In [1]:
# After running
! git clone --recursive https://github.com/Microsoft/LightGBM

#You can run this oneliner which will build and compile LightGBM with GPU enabled in colab:
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;    

Cloning into 'LightGBM'...
remote: Enumerating objects: 25984, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 25984 (delta 71), reused 114 (delta 49), pack-reused 25831
Receiving objects: 100% (25984/25984), 18.93 MiB | 10.28 MiB/s, done.
Resolving deltas: 100% (19162/19162), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21733, done.        
remote: Counting objects: 100% (5/5), done.        
remote: Comp

In [2]:
# 试试GPU 
from lightgbm import LGBMClassifier
from sklearn.datasets import make_moons

model = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=- 1, learning_rate=0.1, n_estimators=300, device_type = "gpu")

train, label = make_moons(n_samples=300000, shuffle=True, noise=0.3, random_state=None)

model.fit(train, label)

[LightGBM] [Info] Number of positive: 150000, number of negative: 150000
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 300000, number of used features: 2
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 2 dense feature groups (1.14 MB) transferred to GPU in 0.002137 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


LGBMClassifier(device_type='gpu', n_estimators=300)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

!pwd

Mounted at /content/drive
/content


In [4]:
import os
os.chdir(r'/content/drive/Othercomputers/我的笔记本电脑/竞赛1/')
os.getcwd()

'/content/drive/Othercomputers/我的笔记本电脑/竞赛1'

In [5]:
import lightgbm as lgbm
lgbm.__version__
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import pandas as pd

In [6]:
# 读取数据
data=pd.read_csv("data/data_2.csv",index_col=0)
# userID和ffID升序排列，Similarity降序
tempp = data.sort_values(['userID','Similarity','ffID'],ascending=[1,0,1])
# 每个userID前30条数据，共241*30+1条
test = tempp.groupby(['userID']).head(60)
# del tempp
data_train=data.drop(test.index,axis=0) # 被删之后的数据
test = tempp.groupby(['userID']).head(30)
del tempp
# data_train=data_train.sample(frac=0.6,axis=0)
# tobedeleted
# test=pd.read_csv("")
display(data,data_train,test)

,userID,ffID,cn,targetUser_x,fName_x,gender_x,starUser_x,mobileUser_x,homeProvince_x,homeCity_x,...,privacy_y,timeline_y,isFriend,HPI,Jaccard,Sorenson,scaled_HPI,scaled_Jaccard,scaled_Sorenson,Similarity
index,,,,,,,,,,,,,,,,,,,,,
0,87702431,87656949,1,1,王,1.0,1.0,0.0,江西,九江市,...,1.0,0,0,0.000677,0.000553,0.001105,0.001008,0.000274,0.000821,0.000701
1,87808217,87656949,10,1,张,1.0,1.0,1.0,陕西,西安市,...,1.0,0,0,0.006770,0.004537,0.009033,0.010148,0.002266,0.006768,0.006394
2,139840069,87656949,1,1,叶,0.0,1.0,0.0,安徽,黄山市,...,1.0,0,0,0.000677,0.000552,0.001103,0.001008,0.000273,0.000819,0.000700
3,150023465,87656949,1,1,沈,1.0,1.0,0.0,安徽,NaN,...,1.0,0,0,0.000677,0.000549,0.001096,0.001008,0.000272,0.000815,0.000698
4,155712967,87656949,1,1,吕,1.0,0.0,0.0,河南,商丘市,...,1.0,0,0,0.000677,0.000521,0.001042,0.001008,0.000258,0.000774,0.000680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205669,3195846773,1516349333,18,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,0,0.030769,0.028125,0.054711,0.046147,0.014060,0.041026,0.033744
1205674,3195846773,3290351601,24,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,0,0.041026,0.027972,0.054422,0.061531,0.013984,0.040809,0.038775
1205676,3195846773,3340133093,18,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,0,0.030769,0.022059,0.043165,0.046147,0.011027,0.032367,0.029847


,userID,ffID,cn,targetUser_x,fName_x,gender_x,starUser_x,mobileUser_x,homeProvince_x,homeCity_x,...,privacy_y,timeline_y,isFriend,HPI,Jaccard,Sorenson,scaled_HPI,scaled_Jaccard,scaled_Sorenson,Similarity
index,,,,,,,,,,,,,,,,,,,,,
0,87702431,87656949,1,1,王,1.0,1.0,0.0,江西,九江市,...,1.0,0,0,0.000677,0.000553,0.001105,0.001008,0.000274,0.000821,0.000701
1,87808217,87656949,10,1,张,1.0,1.0,1.0,陕西,西安市,...,1.0,0,0,0.006770,0.004537,0.009033,0.010148,0.002266,0.006768,0.006394
2,139840069,87656949,1,1,叶,0.0,1.0,0.0,安徽,黄山市,...,1.0,0,0,0.000677,0.000552,0.001103,0.001008,0.000273,0.000819,0.000700
3,150023465,87656949,1,1,沈,1.0,1.0,0.0,安徽,NaN,...,1.0,0,0,0.000677,0.000549,0.001096,0.001008,0.000272,0.000815,0.000698
4,155712967,87656949,1,1,吕,1.0,0.0,0.0,河南,商丘市,...,1.0,0,0,0.000677,0.000521,0.001042,0.001008,0.000258,0.000774,0.000680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205595,1591962673,626202995,5,1,刘,1.0,1.0,1.0,NaN,NaN,...,1.0,0,0,0.010225,0.006840,0.013587,0.015330,0.003417,0.010183,0.009643
1205596,1591962673,763834341,16,1,刘,1.0,1.0,1.0,NaN,NaN,...,1.0,0,0,0.032720,0.021858,0.042781,0.049073,0.010926,0.032078,0.030692
1205597,1591962673,819950357,14,1,刘,1.0,1.0,1.0,NaN,NaN,...,1.0,0,0,0.023850,0.013183,0.026022,0.035768,0.006589,0.019509,0.020622


,userID,ffID,cn,targetUser_x,fName_x,gender_x,starUser_x,mobileUser_x,homeProvince_x,homeCity_x,...,privacy_y,timeline_y,isFriend,HPI,Jaccard,Sorenson,scaled_HPI,scaled_Jaccard,scaled_Sorenson,Similarity
index,,,,,,,,,,,,,,,,,,,,,
206171,87702431,756472073,28,1,王,1.0,1.0,0.0,江西,九江市,...,1.0,0,0,0.084084,0.052434,0.099644,0.126120,0.026215,0.074726,0.075687
1886,87702431,87697903,26,1,王,1.0,1.0,0.0,江西,九江市,...,1.0,1,0,0.078078,0.046099,0.088136,0.117110,0.023047,0.066095,0.068751
207651,87702431,766419155,22,1,王,1.0,1.0,0.0,江西,九江市,...,1.0,0,0,0.066066,0.054187,0.102804,0.099092,0.027091,0.077096,0.067760
19265,87702431,185203523,19,1,王,1.0,1.0,0.0,江西,九江市,...,NaN,0,0,0.057057,0.060510,0.114114,0.085579,0.030252,0.085579,0.067137
217069,87702431,1488502789,25,1,王,1.0,1.0,0.0,江西,九江市,...,1.0,0,0,0.075075,0.044092,0.084459,0.112606,0.022043,0.063338,0.065996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078889,3195846773,548166323,18,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,0,0.021978,0.012987,0.025641,0.032960,0.006491,0.019223,0.019558
1205638,3195846773,551793299,12,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,0,0.020513,0.012632,0.024948,0.030762,0.006313,0.018704,0.018593
354202,3195846773,574972069,10,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,0,0.017094,0.014327,0.028249,0.025634,0.007161,0.021179,0.017991


In [7]:
data2=pd.read_csv("data/data2_2.csv",index_col=0)
display(data2)

,userID,ffID,cn,targetUser_x,fName_x,gender_x,starUser_x,mobileUser_x,homeProvince_x,homeCity_x,...,privacy_y,timeline_y,isFriend,HPI,Jaccard,Sorenson,scaled_HPI,scaled_Jaccard,scaled_Sorenson,Similarity
index,,,,,,,,,,,,,,,,,,,,,
24,549698591,87656949,43,1,楼,0.0,1.0,0.0,浙江,金华市,...,1.0,0,1,0.022549,0.012870,0.025414,0.033815,0.006433,0.019053,0.019767
69,531063051,87660311,6,1,任,1.0,1.0,1.0,NaN,NaN,...,1.0,0,1,0.004866,0.003009,0.006000,0.007292,0.001502,0.004493,0.004429
134,87808217,87661377,28,1,张,1.0,1.0,1.0,陕西,西安市,...,0.0,1,1,0.002000,0.001903,0.003799,0.002992,0.000949,0.002842,0.002261
238,87808217,87661441,57,1,张,1.0,1.0,1.0,陕西,西安市,...,1.0,0,1,0.059686,0.034862,0.067376,0.089522,0.017429,0.050525,0.052492
261,575672873,87661455,3,1,刘,1.0,1.0,1.0,安徽,合肥市,...,1.0,0,1,0.005076,0.003398,0.006772,0.007607,0.001696,0.005072,0.004791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205678,3195846773,3423416727,22,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,1,0.031206,0.017350,0.034109,0.046801,0.008673,0.025574,0.027016
1205679,3195846773,3486811249,22,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,1,0.037607,0.026829,0.052257,0.056403,0.013412,0.039185,0.036333
1205680,3195846773,3575901351,17,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,1,0.029060,0.022049,0.043147,0.043583,0.011022,0.032353,0.028986


In [8]:
# data_train 与data2合并
data_train=pd.concat([data_train,data2],axis=0,ignore_index=True)
data_train

,userID,ffID,cn,targetUser_x,fName_x,gender_x,starUser_x,mobileUser_x,homeProvince_x,homeCity_x,...,privacy_y,timeline_y,isFriend,HPI,Jaccard,Sorenson,scaled_HPI,scaled_Jaccard,scaled_Sorenson,Similarity
0,87702431,87656949,1,1,王,1.0,1.0,0.0,江西,九江市,...,1.0,0,0,0.000677,0.000553,0.001105,0.001008,0.000274,0.000821,0.000701
1,87808217,87656949,10,1,张,1.0,1.0,1.0,陕西,西安市,...,1.0,0,0,0.006770,0.004537,0.009033,0.010148,0.002266,0.006768,0.006394
2,139840069,87656949,1,1,叶,0.0,1.0,0.0,安徽,黄山市,...,1.0,0,0,0.000677,0.000552,0.001103,0.001008,0.000273,0.000819,0.000700
3,150023465,87656949,1,1,沈,1.0,1.0,0.0,安徽,NaN,...,1.0,0,0,0.000677,0.000549,0.001096,0.001008,0.000272,0.000815,0.000698
4,155712967,87656949,1,1,吕,1.0,0.0,0.0,河南,商丘市,...,1.0,0,0,0.000677,0.000521,0.001042,0.001008,0.000258,0.000774,0.000680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190280,3195846773,3423416727,22,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,1,0.031206,0.017350,0.034109,0.046801,0.008673,0.025574,0.027016
1190281,3195846773,3486811249,22,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,1,0.037607,0.026829,0.052257,0.056403,0.013412,0.039185,0.036333
1190282,3195846773,3575901351,17,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,1,0.029060,0.022049,0.043147,0.043583,0.011022,0.032353,0.028986
1190283,3195846773,3818278177,16,1,袁,1.0,1.0,0.0,山西,阳泉市,...,1.0,0,1,0.027350,0.017699,0.034783,0.041018,0.008847,0.026080,0.025315


In [9]:
y_train=data_train.loc[:,"isFriend"]
y_test=test.loc[:,"isFriend"]
display(y_train,y_test)

0          0
1          0
2          0
3          0
4          0
          ..
1190280    1
1190281    1
1190282    1
1190283    1
1190284    1
Name: isFriend, Length: 1190285, dtype: int64

index
206171     0
1886       0
207651     0
19265      0
217069     0
          ..
1078889    0
1205638    0
354202     0
1205604    0
524878     0
Name: isFriend, Length: 7231, dtype: int64

In [10]:
data_train.columns

Index(['userID', 'ffID', 'cn', 'targetUser_x', 'fName_x', 'gender_x',
       'starUser_x', 'mobileUser_x', 'homeProvince_x', 'homeCity_x',
       'nFriends_x', 'nLike_x', 'nPosts_x', 'nShares_x', 'nAlbums_x',
       'nShortPosts_x', 'nVisits_x', 'location_x', 'privacy_x', 'timeline_x',
       'Unnamed: 0_y', 'id_y', 'targetUser_y', 'fName_y', 'gender_y',
       'starUser_y', 'mobileUser_y', 'homeProvince_y', 'homeCity_y',
       'nFriends_y', 'nLike_y', 'nPosts_y', 'nShares_y', 'nAlbums_y',
       'nShortPosts_y', 'nVisits_y', 'location_y', 'privacy_y', 'timeline_y',
       'isFriend', 'HPI', 'Jaccard', 'Sorenson', 'scaled_HPI',
       'scaled_Jaccard', 'scaled_Sorenson', 'Similarity'],
      dtype='object')

In [11]:
data_train.loc[:,['Unnamed: 0_y', 'id_y', 'targetUser_y']]

,Unnamed: 0_y,id_y,targetUser_y
0,6073,87656949,0
1,6073,87656949,0
2,6073,87656949,0
3,6073,87656949,0
4,6073,87656949,0
...,...,...,...
1190280,5830,3423416727,0
1190281,5848,3486811249,0
1190282,5868,3575901351,0
1190283,5914,3818278177,0


In [12]:
# 生成X，不复制data_all,而是维护一个not_X表，不要这个表格里面的列
# userID 不能删，用来区别不同的用户
# targetUser_y 不删， 用来指示是否是网络的中心
not_X=["isFriend","homeProvince_x","homeProvince_y",
       "userID","ffID","targetUser_x",'Unnamed: 0_y', 'id_y','HPI', 
       'Jaccard', 'Sorenson', 'scaled_HPI',
       'scaled_Jaccard', 'scaled_Sorenson', 'Similarity']
category_X=[ "homeCity_x", "location_x","homeCity_y","location_y","fName_y","fName_x"]
X_train=data_train.loc[:,[i for i in data_train.columns if  i not in not_X]]
X_test=test.loc[:,[i for i in test.columns if  i not in not_X]]
for i in category_X:
  X_train[i]=X_train[i].astype("category")
  X_test[i]=X_test[i].astype("category")
display(X_train,X_test)

,cn,fName_x,gender_x,starUser_x,mobileUser_x,homeCity_x,nFriends_x,nLike_x,nPosts_x,nShares_x,...,nFriends_y,nLike_y,nPosts_y,nShares_y,nAlbums_y,nShortPosts_y,nVisits_y,location_y,privacy_y,timeline_y
0,1,王,1.0,1.0,0.0,九江市,333.0,NaN,14.0,NaN,...,1477.0,5.0,24.0,NaN,1.0,78.0,6719.0,清华大学,1.0,0
1,10,张,1.0,1.0,1.0,西安市,737.0,27.0,4.0,NaN,...,1477.0,5.0,24.0,NaN,1.0,78.0,6719.0,清华大学,1.0,0
2,1,叶,0.0,1.0,0.0,黄山市,337.0,NaN,12.0,NaN,...,1477.0,5.0,24.0,NaN,1.0,78.0,6719.0,清华大学,1.0,0
3,1,沈,1.0,1.0,0.0,NaN,347.0,NaN,0.0,NaN,...,1477.0,5.0,24.0,NaN,1.0,78.0,6719.0,清华大学,1.0,0
4,1,吕,1.0,0.0,0.0,商丘市,443.0,1.0,25.0,NaN,...,1477.0,5.0,24.0,NaN,1.0,78.0,6719.0,清华大学,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190280,22,袁,1.0,1.0,0.0,阳泉市,585.0,5.0,1.0,NaN,...,705.0,1.0,2.0,NaN,1.0,1.0,466.0,吉林大学,1.0,0
1190281,22,袁,1.0,1.0,0.0,阳泉市,585.0,5.0,1.0,NaN,...,257.0,2.0,0.0,NaN,0.0,1.0,324.0,吉林大学,1.0,0
1190282,17,袁,1.0,1.0,0.0,阳泉市,585.0,5.0,1.0,NaN,...,203.0,NaN,0.0,NaN,0.0,3.0,277.0,吉林大学,1.0,0
1190283,16,袁,1.0,1.0,0.0,阳泉市,585.0,5.0,1.0,NaN,...,335.0,NaN,3.0,NaN,3.0,7.0,756.0,吉林大学,1.0,0


,cn,fName_x,gender_x,starUser_x,mobileUser_x,homeCity_x,nFriends_x,nLike_x,nPosts_x,nShares_x,...,nFriends_y,nLike_y,nPosts_y,nShares_y,nAlbums_y,nShortPosts_y,nVisits_y,location_y,privacy_y,timeline_y
index,,,,,,,,,,,,,,,,,,,,,
206171,28,王,1.0,1.0,0.0,九江市,333.0,NaN,14.0,NaN,...,229.0,3.0,0.0,NaN,0.0,1.0,454.0,昆明市,1.0,0
1886,26,王,1.0,1.0,0.0,九江市,333.0,NaN,14.0,NaN,...,257.0,NaN,NaN,NaN,NaN,NaN,489.0,NaN,1.0,1
207651,22,王,1.0,1.0,0.0,九江市,333.0,NaN,14.0,NaN,...,95.0,NaN,0.0,NaN,0.0,0.0,12.0,美国,1.0,0
19265,19,王,1.0,1.0,0.0,九江市,333.0,NaN,14.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
217069,25,王,1.0,1.0,0.0,九江市,333.0,NaN,14.0,NaN,...,259.0,NaN,0.0,NaN,1.0,12.0,531.0,德国,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078889,18,袁,1.0,1.0,0.0,阳泉市,585.0,5.0,1.0,NaN,...,819.0,1.0,5.0,NaN,8.0,10.0,1575.0,北方民大,1.0,0
1205638,12,袁,1.0,1.0,0.0,阳泉市,585.0,5.0,1.0,NaN,...,377.0,22.0,37.0,NaN,3.0,36.0,568.0,北京交通大学,1.0,0
354202,10,袁,1.0,1.0,0.0,阳泉市,585.0,5.0,1.0,NaN,...,123.0,2.0,0.0,NaN,1.0,4.0,80.0,九江学院,1.0,0


In [ ]:
clf=lgbm.LGBMClassifier(device_type="gpu",subsample_for_bin=1185)
clf=clf.fit(X_train,y_train)
feature_importance=clf.feature_importances_
ind_feature=np.argsort(feature_importance)[::-1]
importance={X_test.columns[i]:feature_importance[i] for i in ind_feature}


[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Info] Number of positive: 46009, number of negative: 1144276
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 1926
[LightGBM] [Info] Number of data points in the train set: 1190285, number of used features: 32
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 20 dense feature groups (22.70 MB) transferred to GPU in 0.032491 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.038654 -> initscore=-3.213690
[LightGBM] [Info] Start training from score -3.213690


In [ ]:
params={
    # "num_leaves":np.arange(20,100,20),
        "n_estimators":[100,500,1000,1500],
        # "subsample_for_bin":[200000,300000],
        # "max_depth":[-1,5,50,100],
        # "learning_rate":[0.01,0.05,0.1],
        # "n_estimators":[80,100,150,300,1000],
        # "min_split_gain":[0.,1e-3,1e-5,0.1],
        # "reg_alpha":[0.,1e-3,0.01,0.1,1],
        "reg_lambda":[0.,1e-3,0.01,0.1,1]
        # "subsample_freq":[0,0.1,0.01]
        }
clf=lgbm.LGBMClassifier(device_type="gpu",subsample_for_bin=1185)
cv=RandomizedSearchCV(clf,params,scoring="roc_auc",n_iter=5,refit=True,random_state=0)
cv.fit(X_train,y_train)
clf=cv.best_estimator_

[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Info] Number of positive: 36808, number of negative: 915420
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 1878
[LightGBM] [Info] Number of data points in the train set: 952228, number of used features: 32
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 20 dense feature groups (18.16 MB) transferred to GPU in 0.026771 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.038655 -> initscore=-3.213668
[LightGBM] [Info] Start training from score -3.213668
[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Info] Number of positi

In [16]:
feature_importance=clf.feature_importances_
ind_feature=np.argsort(feature_importance)[::-1]
# importance={X_test.columns[i]:feature_importance[i] for i in ind_feature}
importance={X_test.columns[i]:feature_importance[i] for i in ind_feature}
importance

{'cn': 435,
 'fName_x': 514,
 'fName_y': 97,
 'gender_x': 7,
 'gender_y': 8,
 'homeCity_x': 237,
 'homeCity_y': 226,
 'location_x': 161,
 'location_y': 217,
 'mobileUser_x': 0,
 'mobileUser_y': 0,
 'nAlbums_x': 21,
 'nAlbums_y': 10,
 'nFriends_x': 159,
 'nFriends_y': 287,
 'nLike_x': 46,
 'nLike_y': 5,
 'nPosts_x': 14,
 'nPosts_y': 2,
 'nShares_x': 0,
 'nShares_y': 0,
 'nShortPosts_x': 16,
 'nShortPosts_y': 31,
 'nVisits_x': 45,
 'nVisits_y': 25,
 'privacy_x': 5,
 'privacy_y': 0,
 'starUser_x': 0,
 'starUser_y': 1,
 'targetUser_y': 26,
 'timeline_x': 5,
 'timeline_y': 0}

In [19]:
clf.get_params

<bound method LGBMModel.get_params of LGBMClassifier(device_type='gpu', learning_rate=0.05, max_depth=5,
               min_split_gain=0.001, num_leaves=27, reg_alpha=0.1,
               reg_lambda=0.1, subsample_for_bin=1185)>

In [ ]:
display(clf.params_)
clf.booster_.save_model("data/booster_.pt")

In [20]:
# 预测
proba=clf.predict_proba(X_test)
result=test.copy()
result.loc[:,"clf_proba"]=proba[:,1]
# data.sort_values(['userID','Similarity','ffID'],ascending=[1,0,1])
result=result.sort_values(['userID','clf_proba','ffID'],ascending=[1,0,1])
# 每个userID前30条数据，共241*30+1条
# test = tempp.groupby(['userID']).head(30)
result.to_csv("data/result.csv")

In [21]:
result["clf_proba"]

index
206171     0.983053
25521      0.982413
1886       0.981003
116167     0.980943
217069     0.978610
             ...   
1205677    0.779160
1205661    0.775303
1205604    0.720960
1205638    0.673772
524878     0.610843
Name: clf_proba, Length: 7231, dtype: float64

In [ ]:
test

,userID,ffID,cn,targetUser_x,gender_x,starUser_x,mobileUser_x,nFriends_x,nLike_x,nPosts_x,...,Sorenson,scaled_HPI,scaled_Jaccard,scaled_Sorenson,Similarity,fName,homeProvince,homeCity,location,clf_proba
index,,,,,,,,,,,,,,,,,,,,,
206171,87702431,756472073,28,1,1.0,1.0,0.0,333.0,NaN,14.0,...,0.099644,0.126120,0.026215,0.074726,0.075687,0.000255,0,0,0,0.992544
1886,87702431,87697903,26,1,1.0,1.0,0.0,333.0,NaN,14.0,...,0.088136,0.117110,0.023047,0.066095,0.068751,0.000000,0,0,0,0.992542
207651,87702431,766419155,22,1,1.0,1.0,0.0,333.0,NaN,14.0,...,0.102804,0.099092,0.027091,0.077096,0.067760,0.000000,0,0,0,0.992524
19265,87702431,185203523,19,1,1.0,1.0,0.0,333.0,NaN,14.0,...,0.114114,0.085579,0.030252,0.085579,0.067137,0.000000,0,0,0,0.979886
217069,87702431,1488502789,25,1,1.0,1.0,0.0,333.0,NaN,14.0,...,0.084459,0.112606,0.022043,0.063338,0.065996,0.000000,0,0,0,0.992505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078889,3195846773,548166323,18,1,1.0,1.0,0.0,585.0,5.0,1.0,...,0.025641,0.032960,0.006491,0.019223,0.019558,0.000000,1,1,0,0.870574
1205638,3195846773,551793299,12,1,1.0,1.0,0.0,585.0,5.0,1.0,...,0.024948,0.030762,0.006313,0.018704,0.018593,0.000000,0,0,0,0.802155
354202,3195846773,574972069,10,1,1.0,1.0,0.0,585.0,5.0,1.0,...,0.028249,0.025634,0.007161,0.021179,0.017991,0.000000,1,1,0,0.911834


In [ ]:
pd.unique(test.userID).shape

(242,)

In [ ]:
7231/242

29.880165289256198